ДЗ 1. Тесты NIST

In [1]:
def test_1_bits(lst): # Частотный побитовый тест
    n = len(lst)
    s = sum(1 if num == 1 else -1 for num in lst) #сумма 
    #формула
    s_obs = abs(s) / math.sqrt(n)
    p_value = math.erfc(s_obs / math.sqrt(2))
    print(f"p-value: {p_value}")
    return p_value > 0.01

In [2]:
import random
import math
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_1_bits(lst))

p-value: 0.841480581121794
True


In [3]:
def test_2_block(lst, m):
    n = len(lst)
    if n < m:  # если блок больше длины списка
        print("Размер блока больше длины списка!")
        return False
    # Количество блоков, на которые делим список
    num_blocks = n // m
    # Пропорции единиц в каждом блоке
    proportions = []
    for i in range(num_blocks):
        block = lst[i * m : (i + 1) * m]
        # Рассчет пропорции единиц в блоке
        prop = sum(block) / m
        proportions.append(prop)
    # Рассчет Xi-квадрата
    chi_square = 4 * m * sum((prop - 0.5) ** 2 for prop in proportions)
    # Рассчет p-value
    p_value = math.erfc(math.sqrt(chi_square / 2))
    print(f"p-value: {p_value}")
    return p_value > 0.01

In [4]:
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_2_block(lst, m = 6))

p-value: 0.0
False


In [5]:
def test_3_runs(lst):
    """
    подсчёт полного числа рядов в исходной последовательности,
    где ряд — это непрерывная подпоследовательность одинаковых битов.

    """
    n = len(lst)
    
    # Вычисление пропорции единиц в последовательности
    pi = sum(lst) / n
    
    # Проверка, что пропорция единиц близка к 0.5
    if abs(pi - 0.5) > (2 / math.sqrt(n)):
        return False
    # Подсчет количества рядов
    v_obs = sum(lst[i] != lst[i + 1] for i in range(n - 1)) + 1
    # Формула для p-value
    p_value = math.erfc(abs(v_obs - 2 * n * pi * (1 - pi)) / (2 * math.sqrt(2 * n) * pi * (1 - pi)))
    print(f"p-value: {p_value}")
    return p_value > 0.01


In [6]:
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_3_runs(lst))

p-value: 0.07157754135604434
True


In [7]:
import math
import random
from itertools import groupby

def test_4_longest_ones(lst, m):
    """
    Тест на длину самого длинного ряда единиц в блоках
    """
    n = len(lst)
    num_blocks = n // m
    
    # Определение максимальной длины рядов единиц в каждом блоке
    max_runs = []
    for i in range(num_blocks):
        block = lst[i*m:(i+1)*m]
        # Определение максимальной длины ряда единиц в текущем блоке
        max_run_length = 0
        for k, g in groupby(block):
            if k == 1:
                max_run_length = max(max_run_length, len(list(g)))
        max_runs.append(max_run_length)
    if not max_runs:
        print("No valid blocks found.")
        return False
    # максимальное значение
    observed_max = max(max_runs)
    # Ожидаемое значение
    expected = math.log2(m)
    p_value = math.erfc((observed_max - expected) / math.sqrt(2 * expected))
    print(f"p-value: {p_value}")
    return p_value > 0.01

In [8]:
from itertools import groupby
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_4_longest_ones(lst, m = 4))

p-value: 0.15729920705028513
True


In [9]:
def test_5_rank(lst):
    """
    Тест рангов бинарных матриц

    В этом тесте вычисляется ранг непересекающихся подматриц,
    построенных из исходной двоичной последовательности.
    Проверка на линейную зависимость подстрок фиксированной длины,
    составляющих первоначальную последовательность.
    """
    n = len(lst)
    matrix_size = 32  # Размер матрицы
    num_matrices = n // (matrix_size * matrix_size)  # Количество матриц, которые можно сформировать

    # Формирование списка матриц из бинарной последовательности
    matrices = [
        np.reshape(lst[i*matrix_size*matrix_size:(i+1)*matrix_size*matrix_size], (matrix_size, matrix_size))
        for i in range(num_matrices)
    ]

    # Вычисление ранга каждой матрицы
    ranks = [np.linalg.matrix_rank(matrix) for matrix in matrices]

    # Подсчет количества матриц полного ранга
    full_rank = ranks.count(matrix_size)

    # Ожидаемое количество матриц
    expected_full_rank = 0.2888 * num_matrices

    # Рассчет хи-квадрат
    chi_square = ((full_rank - expected_full_rank) ** 2) / expected_full_rank

    # Рассчет p-value
    p_value = math.erfc(math.sqrt(chi_square / 2))
    print(f"p-value: {p_value}")
    return p_value > 0.01

In [10]:
import numpy as np
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_5_rank(lst))

p-value: 7.18067510581265e-05
False


In [11]:
import numpy as np
import math
import random

def test_6_dft(lst):
    """
    Тест на дискретное преобразование Фурье (DFT)
    Этот тест оценивает высоту пиков дискретного преобразования Фурье 
    бинарной последовательности. Цель — выявление периодических свойств
    последовательности, таких как повторяющиеся участки.
    """
    n = len(lst)
    # Преобразуем бинарные значения (0 и 1) в значения (-1 и 1)
    bit_sequence = [2 * bit - 1 for bit in lst]
    # Выполняем дискретное преобразование Фурье
    fft_result = np.fft.fft(bit_sequence)
    # Вычисляем амплитуды первых половин преобразования (симметрия)
    magnitudes = np.abs(fft_result)[:n//2]
    # Определяем пороговое значение для пиков
    threshold = math.sqrt(2.995732274 * n)
    num_peaks = sum(mag > threshold for mag in magnitudes)
    # Ожидаемое количество пиков, превышающих порог
    expected_peaks = 0.95 * n / 2
    # Рассчитываем p-value
    std_dev = math.sqrt(0.95 * 0.05 * n / 2)  # Стандартное отклонение
    p_value = math.erfc(abs(num_peaks - expected_peaks) / std_dev)
    print(f"p-value: {p_value}")
    return p_value > 0.01

In [12]:
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_6_dft(lst))

p-value: 0.0
False


In [13]:
def test_7_non_overlapping(lst, template):
    """
    Тест на совпадение неперекрывающихся шаблонов

    В этом тесте подсчитывается количество неперекрывающихся вхождений заранее определенного шаблона
    в бинарной последовательности. Цель — обнаружение генераторов случайных чисел, которые
    формируют шаблоны слишком часто.
    """
    n = len(lst)
    template_len = len(template)
    matches = 0
    # Поиск неперекрывающихся вхождений шаблона в последовательности
    for i in range(n - template_len + 1):
        if lst[i:i+template_len] == template:
             # Перемещаем окно на размер шаблона вперед (без перекрытия)
            matches += 1
    # Ожидаемое количество вхождений шаблона
    expected_matches = (n - template_len + 1) / (2 ** template_len)
    p_value = math.erfc(abs(matches - expected_matches) / math.sqrt(2 * expected_matches))
    print(f"p-value: {p_value}")
    return p_value > 0.01

In [14]:
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_7_non_overlapping(lst, [1, 0, 1]))

p-value: 0.676506727070678
True


In [15]:
def test_8_overlapping(lst, template):
    """
    Тест на совпадение перекрывающихся шаблонов

    Этот тест подсчитывает количество перекрывающихся вхождений заранее определенного шаблона
    в бинарной последовательности. В отличие от теста на неперекрывающиеся шаблоны,
    здесь окно перемещается на один бит вперед даже если шаблон найден.
    """
    n = len(lst)
    template_len = len(template)
    matches = 0
    
    # Перебираем все возможные позиции шаблона в последовательности
    for i in range(n - template_len + 1):
        if lst[i:i+template_len] == template:
            matches += 1

    # Рассчитываем ожидаемое количество вхождений шаблона
    expected_matches = (n - template_len + 1) / (2 ** template_len)
    
    # Рассчитываем дисперсию количества вхождений
    variance = n * (1 / (2 ** template_len)) * (1 - 1 / (2 ** template_len))
    chi_square = abs(matches - expected_matches) / math.sqrt(2 * variance)
    p_value = math.erfc(chi_square)
    print(f"p-value: {p_value}")
    return p_value > 0.01

In [16]:
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_8_overlapping(lst, [1, 0, 1]))

p-value: 0.7451436487037852
True


In [17]:
def test_9_maurer(lst):
    """
    Тест на случайность по методу Мауэра

    Этот тест проверяет случайность бинарной последовательности,
    основываясь на логарифмах разностей позиций появления
    бинарных блоков фиксированной длины.
    """
    n = len(lst)
    L = 6  # Длина блока
    Q = 10 * (2 ** L)  # Количество блоков для инициализации
    K = n // L - Q  # Количество блоков для тестирования
    T = [-1] * (2 ** L)  # Инициализация T со значениями -1
    sum_val = 0.0
    for i in range(Q):
        T[int(''.join(map(str, lst[i * L:(i + 1) * L])), 2)] = i + 1
    for i in range(Q, Q + K):
        # Преобразование текущего блока в десятичное представление
        dec_rep = int(''.join(map(str, lst[i * L:(i + 1) * L])), 2)
        if T[dec_rep] != -1:
            # Рассчитываем логарифм разности
            diff = i - T[dec_rep]
            if diff > 0:
                sum_val += math.log2(diff)
        T[dec_rep] = i + 1
    expected_value = 7.1836656
    variance = 3.238
    p_value = math.erfc(abs(sum_val / K - expected_value) / (math.sqrt(variance / K)))
    print(f"p-value: {p_value}")
    return p_value > 0.01

In [18]:
lst = []
for i in range(100000):
    lst.append(random.randint(0, 1))
print(test_9_maurer(lst))

p-value: 0.0
False


In [19]:
def test_10_lin_comp(lst, seq_size):
    n = len(lst)
    if n < seq_size:
        return False
    
    num_seqs = n // seq_size
    complexities = []

    # Подсчет линейной сложности для каждого блока
    for i in range(num_seqs):
        block = lst[i * seq_size:(i + 1) * seq_size]
        complexities.append(berlekamp_massey_algorithm(block))  # Используем алгоритм Берлекэмпа-Масси

    # Средняя сложность
    avg_complexity = sum(complexities) / num_seqs
    expected_complexity = seq_size / 2 + (9 + (-1) ** seq_size) / 36 - (seq_size / 3 + 2 / 9) / seq_size
    p_value = math.erfc(abs(avg_complexity - expected_complexity) / (math.sqrt(2 * seq_size ** 2 / 45)))
    print(f"p-value: {p_value}")
    return p_value > 0.01

def berlekamp_massey_algorithm(lst):
    # Алгоритм Берлекэмпа-Масси для оценки линейной сложности
    n = len(lst)
    c = [0] * n
    b = [0] * n
    c[0] = 1
    b[0] = 1
    l, m, i = 0, -1, 0
    for i in range(n):
        discrepancy = lst[i]
        for j in range(1, l + 1):
            discrepancy ^= (c[j] * lst[i - j])
        if discrepancy == 1:
            t = c[:]
            for j in range(n - i + m):
                c[i - m + j] ^= b[j]
            if l <= i // 2:
                l = i + 1 - l
                m = i
                b = t
    return l

In [20]:
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_10_lin_comp(lst, 500))

p-value: 0.9972595907454542
True


In [21]:
def test_11_serial(lst, m):
    """
    Тест на периодичность.
    Проверяет частоту появления всех возможных шаблонов длиной m бит в последовательности.
    Цель — определить, действительно ли количество появлений всех возможных шаблонов длиной m бит
    приблизительно такое же, как в случае абсолютно случайной входной последовательности бит.
    """
    n = len(lst)
    if n < m:  # Проверка, чтобы длина последовательности была больше длины шаблона
        return False
    counts = {}
    
    # Проходим по всем возможным шаблонам длиной m
    for i in range(n - m + 1):
        pattern = tuple(lst[i:i+m])  # Извлекаем шаблон длиной m бит
        counts[pattern] = counts.get(pattern, 0) + 1  # Обновляем счетчик для текущего шаблона
    
    # Расчёт среднего и стандартного отклонения
    num_patterns = 2 ** m  # Общее количество возможных шаблонов m бит
    expected_count = (n - m + 1) / num_patterns  # Ожидаемое количество появлений
    variance = (n - m + 1) * (1 - 1 / num_patterns)
    #хи-квадрат
    chi_square = sum((count - expected_count) ** 2 / variance for count in counts.values())
    p_value = math.erfc(chi_square / (2 * math.sqrt(variance)))
    print(f"p-value: {p_value}")
    return p_value > 0.01


In [22]:
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_11_serial(lst, 2))

p-value: 0.9972089066466684
True


In [23]:
def test_12_approximate_entropy(lst, m):
    """
    Тест приблизительной энтропии.

    Сравнивает частоты появления шаблонов длиной m бит и m+1 бит в последовательности,
    чтобы определить, насколько частоты перекрытий отличаются от ожидаемых в абсолютно случайной последовательности.
    """
    n = len(lst)
    if n < m + 1:
        return False
    counts_m = defaultdict(int)
    for i in range(n - m):
        pattern_m = tuple(lst[i:i+m])
        counts_m[pattern_m] += 1
    # Подсчет количества вхождений шаблонов длиной m бит   
    counts_m1 = defaultdict(int)
    for i in range(n - m - 1):
        # Извлечение шаблона длиной m бит
        pattern_m1 = tuple(lst[i:i+m+1])
        counts_m1[pattern_m1] += 1
    num_patterns_m = 2 ** m
    num_patterns_m1 = 2 ** (m + 1)
    
    expected_count_m = (n - m) / num_patterns_m
    expected_count_m1 = (n - m - 1) / num_patterns_m1
    
    # Расчет дисперсии
    variance_m = (n - m) * (1 - 1 / num_patterns_m)
    variance_m1 = (n - m - 1) * (1 - 1 / num_patterns_m1)
    
    #хи-квадрат 
    chi_square_m = sum((count - expected_count_m) ** 2 / variance_m for count in counts_m.values())
    chi_square_m1 = sum((count - expected_count_m1) ** 2 / variance_m1 for count in counts_m1.values())
    
    # Общая статистика
    chi_square_total = chi_square_m + chi_square_m1
    
    # Вычисление p-value
    p_value = math.erfc(chi_square_total / (2 * math.sqrt(variance_m + variance_m1)))
    print(f"p-value: {p_value}")
    
    # Возвращаем True если p-value > 0.01, иначе False
    return p_value > 0.01


In [24]:
from collections import defaultdict
lst = []
for i in range(10000):
    lst.append(random.randint(0, 1))
print(test_12_approximate_entropy(lst, 2))

p-value: 0.9756329644717346
True
